In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd
import os
import sys
import seaborn as sns
import re

sys.path.append("/home/ach94/scripts/")
# sys.path.append("/home/ach94/scripts/spellock")
# import kinetics

from pathlib import Path
import json
from Bio import SeqIO
import scipy.stats as stat
from scipy.stats import linregress

In [ ]:
def parse_mscarlett_and_kinetics(filepath, start_row=54):
    """
    parse a tab delimited plate reader text fuke from synergy neo2 with mscarlett and kinetic reads in the same file.

    inputs
    filepath (string) - string specifying the filepath
    start_row (int) - start row for the mscarlett data

    outputs
    df_mscarlett (dataframe) - dataframe containing mscarlet reads in tidy format
    df_kinetics (dataframe) - dataframe containing kinetic reads in tidy format

    """

    ## parse replicate number
    # Use regex to find the substring "plate" followed by a number
    match = re.search(r"plate(\d+)", filepath)

    # If a match is found, extract the value next to "plate"
    if match:
        replicate_number = int(match.group(1))
    else:
        print(
            "No replicate number found, be sure to include sample# in the file name where # is the replicate number"
        )

    ## parse experiment ID

    ## parse plate ID
    match = re.search(r"sample(\d+)", filepath)

    # If a match is found, extract the value next to "sample"
    if match:
        sample_number = int(match.group(1))
    else:
        print(
            "No plate number found, be sure to include plate# in the file name where # is the plate number"
        )

    # print(f"parsing plate {sample_number} replicate {replicate_number}")

    ## parse instrument serial number
    df = pd.read_csv(
        filepath,
        encoding="cp1252",
        delimiter="\t",
        nrows=20,
    )
    serial_number = df[df.iloc[:, 0] == "Reader Serial Number:"]
    serial_number = serial_number.iloc[:, 1].values[0]

    ## parse mscarlett data
    df_mscarlett = pd.read_csv(
        filepath,
        encoding="cp1252",
        delimiter="\t",
        skiprows=range(1, start_row),
        nrows=1,
    )

    # make dataframe tidy
    df_mscarlett = df_mscarlett.melt(
        id_vars=["Well"], var_name="well", value_name="value"
    )
    df_mscarlett.dropna(inplace=True)

    # clean the naming
    df_mscarlett.drop(columns=["Well"], inplace=True)

    # add info
    df_mscarlett["serial_number"] = serial_number
    # df_mscarlett["replicate_number"] = replicate_number
    # df_mscarlett["sample_number"] = sample_number

    ## parse kinetics data
    df_kinetics = pd.read_csv(
        filepath,
        encoding="cp1252",
        delimiter="\t",
        # header=0,
        skiprows=range(1, start_row + 4),
    )

    df_kinetics = df_kinetics.drop(df_kinetics.columns[1], axis=1)
    df_kinetics.rename(columns={"Time": "time"}, inplace=True)

    # time conversion
    df_kinetics["time"] = pd.to_datetime(
        df_kinetics["time"], format="%H:%M:%S", errors="coerce"
    ).dt.time

    # Convert the time column to seconds
    df_kinetics["time"] = df_kinetics["time"].apply(
        lambda x: x.hour * 3600 + x.minute * 60 + x.second
    )

    # make dataframe tidy
    df_kinetics = df_kinetics.melt(
        id_vars=["time"], var_name="well", value_name="value"
    )
    df_kinetics.dropna(inplace=True)

    # add info
    df_kinetics["serial_number"] = serial_number
    # df_kinetics["replicate_number"] = replicate_number
    # df_kinetics["sample_number"] = sample_number

    # clean dataframe data types
    df_kinetics["value"] = pd.to_numeric(df_kinetics["value"], errors="coerce")

    return df_mscarlett, df_kinetics


# Read the FASTA file and convert it to a DataFrame
def fasta_to_dataframe(fasta_file):
    records = SeqIO.parse(fasta_file, "fasta")
    data = [(record.id, str(record.seq)) for record in records]
    df = pd.DataFrame(data, columns=["ID", "sequence"])
    return df


def fit_line(group, x_column="time", y_column="4MU_um"):
    slope, intercept, r_value, p_value, std_err = linregress(
        group[x_column], group[y_column]
    )
    return pd.Series(
        {
            "rate": slope,
            "intercept": intercept,
            "r_value": r_value,
            "p_value": p_value,
            "std_err": std_err,
        }
    )


# def subtract_background(group):
#     background = group.loc[group["sample_type"] == "negative_control", "rate"].values[0]
#     group["rate_minus_background"] = group["rate"] - background
#     return group


def set_plt_size(w, h, ax=None):
    """w, h: width, height in inches"""
    # Allows for specification of matplotlib figure area (excluding axes labels etc.)
    if not ax:
        ax = plt.gca()
    l = ax.figure.subplotpars.left
    r = ax.figure.subplotpars.right
    t = ax.figure.subplotpars.top
    b = ax.figure.subplotpars.bottom
    figw = float(w) / (r - l)
    figh = float(h) / (t - b)
    ax.figure.set_size_inches(figw, figh)


def parity_plot(reps, params):

    from matplotlib.ticker import FormatStrFormatter

    alpha, ax_label, stat_val = params

    # Set scale for plot and calculate parity line
    n = len(reps)

    max_val = []
    min_val = []
    for i in range(n):
        max_val.append(max(reps[i]))
        min_val.append(min(reps[i]))

    data_len = np.absolute(max(max_val) - min(min_val))
    # print(data_len)
    parity = [min(min_val) - 0.05 * data_len, max(max_val) + 0.05 * data_len]

    # plotting fluorescence agains eachother
    fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 12))

    # plot replicate data
    ax.scatter(reps[0], reps[1], color="black", alpha=alpha)
    # turn minor ticks off, since it is somehow impossible to turn minor x ticks on
    #     ax.tick_params(axis='y', which='minor', left=False)

    # parity=[min([ax.get_xlim()[0],ax.get_ylim()[0]]),max([ax.get_xlim()[1],ax.get_ylim()[1]])]

    # Plot parity line
    ax.plot(parity, parity, "k-", lw=1)

    # Set axis labeles to be the same
    ax.set_aspect("equal", "box")
    # ax.yaxis.set_ticks(ax.get_xticks())
    # ax.xaxis.set_ticks(ax.get_yticks())

    # Ensure the parity line is the limits
    ax.set_xlim(parity)
    ax.set_ylim(parity)

    # Calculate stats
    r = stat.pearsonr(reps[0], reps[1])[0]
    [m, b, R, p, std_err] = stat.linregress(reps[0], reps[1])

    # calculate the postion (in data units) for text
    parity_len = np.absolute(parity[1] - parity[0])
    text_pos = [0.05 * parity_len + parity[0], parity[1] - 0.1 * parity_len]

    # add text to plots
    if stat_val == "r":
        # add pearson correlation
        ax.text(text_pos[0], text_pos[1], "r = " + str(np.round(r, 3)), fontsize=14)
    elif stat_val == "R":
        # to display R**2 value for linear fit, use below
        ax.text(
            text_pos[0], text_pos[1], "R$^{2}$ = " + str(np.round(R**2, 3)), fontsize=14
        )

    # Set axis labels for outside axes
    ax.set_ylabel(ax_label[1])
    ax.set_xlabel(ax_label[0])

    return fig, ax

In [ ]:
### setup the directory
# in what directory is this notebook?
working_directory_path = Path(
    "/home/ach94/projects/p13_iterative_optimization/e197_plate_repeats/"
)

pics_directory = working_directory_path / "pics"
pics_directory.mkdir(exist_ok=True)

In [ ]:
## Import well mappings

df_mappings = pd.read_excel(
    "/home/ach94/projects/p12_small_molecule_acyl_transfer/e200_cys_kinetics/experimental_processed_data/241023_e200_mappings.xlsx"
)

# clean dataframe data types
df_mappings["replicate_number"] = pd.to_numeric(
    df_mappings["replicate_number"], errors="coerce"
)
df_mappings["column"] = pd.to_numeric(df_mappings["column"], errors="coerce")

df_mappings.head()

In [ ]:
temp_dfs = []

In [ ]:
## parse data

filepath = "/home/ach94/projects/p12_small_molecule_acyl_transfer/e200_cys_kinetics/experimental_raw_data/241023_e200_kinetics_1_repeat.txt"
# mscarlett_start_row = 54
kinetics_start_row = 42  # mscarlett_start_row + 4

## parse instrument serial number
df = pd.read_csv(
    filepath,
    encoding="cp1252",
    delimiter="\t",
    nrows=20,
)
serial_number = df[df.iloc[:, 0] == "Reader Serial Number:"]
serial_number = serial_number.iloc[:, 1].values[0]

## parse kinetics data
df_kinetics = pd.read_csv(
    filepath,
    encoding="cp1252",
    delimiter="\t",
    # header=0,
    skiprows=range(1, kinetics_start_row),
)

df_kinetics = df_kinetics.drop(df_kinetics.columns[1], axis=1)
df_kinetics.rename(columns={"Time": "time"}, inplace=True)

# time conversion
df_kinetics["time"] = pd.to_datetime(
    df_kinetics["time"], format="%H:%M:%S", errors="coerce"
).dt.time

# Convert the time column to seconds
df_kinetics["time"] = df_kinetics["time"].apply(
    lambda x: x.hour * 3600 + x.minute * 60 + x.second
)

# make dataframe tidy
df_kinetics = df_kinetics.melt(id_vars=["time"], var_name="well", value_name="value")
df_kinetics.dropna(inplace=True)

# add info
df_kinetics["serial_number"] = serial_number

# clean dataframe data types
df_kinetics["value"] = pd.to_numeric(df_kinetics["value"], errors="coerce")

# add plate number manually
df_kinetics["plate_number"] = 1


temp_dfs.append(df_kinetics)

In [ ]:
df_kinetics = pd.concat(temp_dfs)
df_kinetics[df_kinetics["plate_number"] == 2]

# map data to sample info

df_kinetics = df_kinetics.merge(df_mappings, on=["well", "plate_number"], how="left")

df_kinetics.head()

In [ ]:
# fit the mscarlett and 4MU with the standard curve

standard_curve_json = "/home/ach94/projects/p13_iterative_optimization/e191_kinetic_data_dev/241001_standard_curves.json"

with open(standard_curve_json, "r") as f:
    standard_curves = json.load(f)

df_kinetics["4MU_um"] = df_kinetics.apply(
    lambda x: (x["value"] - standard_curves[x["serial_number"]]["4MU"]["y_intercept"])
    / standard_curves[x["serial_number"]]["4MU"]["slope"],
    axis=1,
)

df_kinetics.head()

In [ ]:
# apply linear fit to 4MU data and process
fit_time_range = (0, 400)

# filter the data to the right time range
df_temp = df_kinetics[
    (df_kinetics["time"] >= fit_time_range[0])
    & (df_kinetics["time"] <= fit_time_range[1])
]

# fit the data with a line
df_kinetics_fits = (
    df_temp.groupby(
        [
            "well",
            "replicate_number",
            "name",
            "sample_type",
            "plate_number",
            "substrate_concentration",
        ]
    )
    .apply(fit_line, x_column="time", y_column="4MU_um")
    .reset_index()
)

df_kinetics_fits.drop(columns=["r_value", "p_value", "std_err"], inplace=True)

# subract background
value = "rate"
grouping = ["plate_number", "substrate_concentration"]
clip = True
epsilon = 1e-8

df_background_rates = (
    df_kinetics_fits[df_kinetics_fits["sample_type"] == "negative_control"]
    .groupby(grouping)[value]
    .mean()
    .reset_index()
    .rename(columns={value: f"background_{value}"})
)

if clip:
    # df_background_rates[f"background_{value}"] = df_background_rates[
    #     f"background_{value}"
    # ].clip(lower=0)

    df_background_rates.loc[
        df_background_rates[f"background_{value}"] < 0, f"background_{value}"
    ] = epsilon

df_kinetics_fits = df_kinetics_fits.merge(df_background_rates, on=grouping, how="left")

df_kinetics_fits[f"{value}_minus_background"] = (
    df_kinetics_fits[value] - df_kinetics_fits[f"background_{value}"]
)

# clip any rates below zero

# df_kinetics_fits[f"rate_minus_background"] = df_kinetics_fits[
#     f"rate_minus_background"
# ].clip(lower=0)

df_kinetics_fits.loc[
    df_kinetics_fits["rate_minus_background"] < 0, "rate_minus_background"
] = 1e-8

# merge all mapping info
grouping = [
    "well",
    "replicate_number",
    "name",
    "sample_type",
    "plate_number",
    "substrate_concentration",
]

df_kinetics_fits = df_kinetics_fits.merge(df_mappings, on=grouping, how="left")


# calculate velocity

df_kinetics_fits["velocity"] = (
    df_kinetics_fits["rate_minus_background"] / df_kinetics_fits["enzyme_concentration"]
)

df_kinetics_fits.head()

In [ ]:
# aggregate kinetics data
grouping = ["name", "plate_number", "substrate_concentration"]

df_kinetics_stats = (
    df_kinetics_fits.groupby(grouping)["velocity"]
    .agg(
        mean="mean",
        std_dev="std",
    )
    .reset_index()
)

df_kinetics_stats.rename(
    columns={
        "mean": "rate_mean",
        "std_dev": "rate_std_dev",
    },
    inplace=True,
)

df_kinetics_stats["rate_cv"] = df_kinetics_stats["rate_std_dev"] / abs(
    df_kinetics_stats["rate_mean"]
)


# grouping = [
#     "name",
#     "substrate_concentration",
#     "plate_number",
# ]

# df_kinetics_stats = df_kinetics_stats.merge(
#     df_mappings[
#         ["name", "substrate_concentration", "sample_type", "plate_number"]
#     ].head(),
#     on=grouping,
#     how="left",
# )

df_kinetics_stats.head()

In [ ]:
df_mappings[["name", "substrate_concentration", "sample_type"]].head()

In [ ]:
df_plot = df_kinetics[df_kinetics["sample_type"] == "sample"]

names = df_plot.name.unique()
num_names = len(names)

# Create subplots
fig, axs = plt.subplots(
    1, num_names, figsize=(num_names * 5, 5), sharex=True, sharey=True
)

# Loop through each unique name to plot
for num, name in enumerate(names):
    sns.lineplot(
        data=df_plot[df_plot["name"] == name],  # Filter data for each replicate
        x="time",
        y="4MU_um",
        hue="substrate_concentration",
        # palette="viridis",
        ax=axs[num] if num_names > 1 else axs,  # Handles single or multiple axes
    )
    axs[num].set_xlabel("Time (seconds)")
    axs[num].set_ylabel("[4MU] (uM)")
    axs[num].set_title(f"{name}")

# Adjust layout for spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
df_kinetics_stats.head()

In [ ]:
from scipy.optimize import curve_fit


# Michaelis-Menten model
def michaelis_menten(s, Vmax, Km):
    return (Vmax * s) / (Km + s)


def fit_michaelis_menten(
    group,
    substrate_concentrations="substrate_concentration",
    velocities="velocity",
    kcat_guess=0.001,
    km_guess=50,
    bounds=(0, [float("inf"), float("inf")]),
):
    # Fit the data
    params, covariance = curve_fit(
        michaelis_menten,
        group[substrate_concentrations],
        group[velocities],
        p0=[kcat_guess, km_guess],
        bounds=bounds,
    )

    # Get the relevant parameters
    kcat_optimized, Km_optimized = params
    kcat_stdev, Km_stdev = np.sqrt(np.diag(covariance))

    # Calculate the uncertainty in kcat/Km by propagating errors
    rel_kcat_unc = kcat_stdev / kcat_optimized
    rel_Km_unc = Km_stdev / Km_optimized
    kcat_Km_unc = (
        kcat_optimized / Km_optimized * np.sqrt(rel_kcat_unc**2 + rel_Km_unc**2)
    )

    return pd.Series(
        {
            "kcat": kcat_optimized,
            "Km": Km_optimized,
            "kcat_stdev": kcat_stdev,
            "Km_stdev": Km_stdev,
            "kcat_Km_uncertainty": kcat_Km_unc,
        }
    )


# velocities


df_params = df_kinetics_stats[df_kinetics_stats["name"] != "negative_control"]

# fit the data with a line
df_params = (
    df_params.groupby(
        [
            "name",
        ]
    )
    .apply(
        fit_michaelis_menten,
        substrate_concentrations="substrate_concentration",
        velocities="rate_mean",
    )
    .reset_index()
)


df_params["efficiency"] = df_params["kcat"] / (df_params["Km"] * 1e-6)

df_params.head()

In [ ]:
df_plot = df_kinetics_stats[df_kinetics_stats["name"] != "negative_control"]

names = df_plot.name.unique()
num_names = len(names)

# Create subplots
fig, axs = plt.subplots(1, num_names, figsize=(num_names * 5, 5))

# Loop through each unique name to plot
for num, name in enumerate(names):
    x = df_plot[df_plot["name"] == name]["substrate_concentration"]
    y = df_plot[df_plot["name"] == name]["rate_mean"]
    axs[num].scatter(x, y)

    kcat = df_params[df_params["name"] == name].kcat.values[0]
    Km = df_params[df_params["name"] == name].Km.values[0]
    x_fit = np.linspace(0, max(x), 100)
    y_fit = michaelis_menten(x_fit, kcat, Km)
    axs[num].plot(x_fit, y_fit)

    axs[num].set_xlabel("[4MU] (uM))")
    axs[num].set_ylabel("Rate/[E] (s$^{-1}$)")
    axs[num].set_title(f"{name}")

# Adjust layout for spacing
plt.tight_layout()

# Show the plot
plt.show()